In [6]:
import pandas as pd
import joblib
from tqdm import tqdm
import os
import time

import pennylane as qml
from pennylane import numpy as np
import pennylane.numpy as pnp

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA

start_time = time.time()

#Load dataset
data = pd.read_csv("mergedTop20.csv")

# Label encoding
label_encoder = LabelEncoder()
data['Label'] = label_encoder.fit_transform(data['Label'])

X = data.drop("Label", axis=1).values
y = data["Label"].values

# PCA → reduce to number of qubits
n_qubits = 4
pca = PCA(n_components=n_qubits)
X_reduced = pca.fit_transform(X)

# Normalize to [0, 2π]
scaler = MinMaxScaler(feature_range=(0, 2 * np.pi))
X_scaled = scaler.fit_transform(X_reduced)

# Map labels from {0,1} → {−1, 1}
y_transformed = 2 * y - 1

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_transformed, test_size=0.2, random_state=42)

# To speed up training
X_train = X_train[:200]
y_train = y_train[:200]

# QUANTUM CIRCUIT
dev = qml.device("lightning.qubit", wires=n_qubits)

def variational_circuit(x, weights):
    qml.AngleEmbedding(x, wires=range(n_qubits), rotation='Y')
    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))

@qml.qnode(dev)
def circuit(x, weights):
    variational_circuit(x, weights)
    return qml.expval(qml.PauliZ(0))

# Prediction
def predict(weights, X):
    preds = [np.sign(circuit(x, weights)) for x in X]
    return np.array([(1 if p >= 0 else -1) for p in preds])

def cost(weights, X, y):
    losses = [qml.math.square(circuit(x, weights) - y_i) for x, y_i in zip(X, y)]
    return qml.math.mean(pnp.stack(losses))

# TRAINING
layers = 1
weights_init = 0.01 * np.random.randn(layers, n_qubits, 3)
opt = qml.AdamOptimizer(stepsize=0.01)
weights = weights_init.copy()

epochs = 30
for epoch in tqdm(range(epochs), desc="Training VQC"):
    weights = opt.step(lambda w: cost(w, X_train, y_train), weights)
    train_preds = predict(weights, X_train)
    acc = accuracy_score((y_train + 1) // 2, (train_preds + 1) // 2)
    tqdm.write(f"Epoch {epoch+1}: Train Accuracy = {acc:.3f}")

#Evaluation
test_preds = predict(weights, X_test)

# Convert predictions from {-1, 1} → {0, 1}
test_preds_orig = (test_preds + 1) // 2
y_test_orig = (y_test + 1) // 2

test_acc = accuracy_score(y_test_orig, test_preds_orig)
print("\nTest Accuracy:", test_acc)
from sklearn.utils.multiclass import unique_labels

all_labels = list(label_encoder.transform(label_encoder.classes_))  # [0, 1, ..., n]
print("\nClassification Report:\n", classification_report(
    y_test_orig, 
    test_preds_orig, 
    labels=all_labels, 
    target_names=label_encoder.classes_
))

save_dir = "Pennylane_Result_vqc"
os.makedirs(save_dir, exist_ok=True)

np.save(os.path.join(save_dir, "vqc_weights.npy"), weights)
joblib.dump(scaler, os.path.join(save_dir, "vqc_scaler.pkl"))
joblib.dump(pca, os.path.join(save_dir, "vqc_pca.pkl"))
joblib.dump(label_encoder, os.path.join(save_dir, "vqc_label_encoder.pkl"))

print("Time taken:", round(time.time() - start_time, 2), "seconds")


Training VQC:   3%|▎         | 1/30 [00:03<01:51,  3.84s/it]

Epoch 1: Train Accuracy = 0.825


Training VQC:   7%|▋         | 2/30 [00:05<01:19,  2.84s/it]

Epoch 2: Train Accuracy = 0.825


Training VQC:  10%|█         | 3/30 [00:08<01:14,  2.75s/it]

...

Training VQC: 100%|██████████| 30/30 [01:20<00:00,  2.67s/it]


Epoch 30: Train Accuracy = 0.825

Test Accuracy: 0.7943332938855319


c:\Users\priya\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\priya\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])



Classification Report:
                             precision    recall  f1-score   support

                    BENIGN       0.83      0.95      0.89    471812
                       Bot       0.00      0.00      0.00       373
                      DDoS       0.00      0.00      0.00      8424
             DoS GoldenEye       0.00      0.00      0.00      1988
                  DoS Hulk       0.00      0.00      0.00     46154
          DoS Slowhttptest       0.00      0.00      0.00      1102
             DoS slowloris       0.00      0.00      0.00      1148
               FTP-Patator       0.00      0.00      0.00      1599
                Heartbleed       0.00      0.00      0.00         3
              Infiltration       0.00      0.00      0.00         7
                  PortScan       0.00      0.00      0.00     31948
               SSH-Patator       0.00      0.00      0.00      1162
  Web Attack - Brute Force       0.00      0.00      0.00       294
Web Attack - Sql Injec

c:\Users\priya\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
